In [2]:
library(randomForest)
library(ranger)
library(stringr)
library(taRifx)
library(mltools)
library(data.table)
library(randomForest)
library(MASS)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘ranger’

The following object is masked from ‘package:randomForest’:

    importance


Attaching package: ‘data.table’

The following objects are masked from ‘package:taRifx’:

    between, first, last, shift



We define a function to calculate our prediction score. Our metric will be Root Mean Logarithmic Squared Error

In [ ]:
rmlse = function(actual, predicted) {
  return(sqrt(mean((log(actual) - log(predicted))^2)))
}

In [3]:
housing_price = read.csv("../input/housing_price.csv",sep=",",header = T)

In [4]:
data = housing_price
data[data == ""]="NA"
summary(housing_price)

Warning message in `[<-.factor`(`*tmp*`, thisvar, value = "NA"):
“invalid factor level, NA generated”Warning message in `[<-.factor`(`*tmp*`, thisvar, value = "NA"):
“invalid factor level, NA generated”Warning message in `[<-.factor`(`*tmp*`, thisvar, value = "NA"):
“invalid factor level, NA generated”Warning message in `[<-.factor`(`*tmp*`, thisvar, value = "NA"):
“invalid factor level, NA generated”Warning message in `[<-.factor`(`*tmp*`, thisvar, value = "NA"):
“invalid factor level, NA generated”

       Id            BATHRM         HF_BATHRM                  HEAT      
 Min.   :    1   Min.   : 0.000   Min.   :0.0000   Forced Air    :15749  
 1st Qu.: 9881   1st Qu.: 1.000   1st Qu.:0.0000   Hot Water Rad :12386  
 Median :19760   Median : 2.000   Median :1.0000   Warm Cool     :10635  
 Mean   :19760   Mean   : 2.127   Mean   :0.7085   Ht Pump       :  508  
 3rd Qu.:29640   3rd Qu.: 3.000   3rd Qu.:1.0000   Water Base Brd:   59  
 Max.   :39520   Max.   :12.000   Max.   :7.0000   Wall Furnace  :   45  
                                                   (Other)       :  138  
 AC            ROOMS           BEDRM             AYB          YR_RMDL     
 0:    5   Min.   : 0.00   Min.   : 0.000   Min.   :1754   Min.   :  20   
 N: 8703   1st Qu.: 6.00   1st Qu.: 3.000   1st Qu.:1917   1st Qu.:2003   
 Y:30812   Median : 7.00   Median : 3.000   Median :1931   Median :2008   
           Mean   : 7.11   Mean   : 3.305   Mean   :1937   Mean   :2005   
           3rd Qu.: 8.00   3rd Qu

In [5]:
summary(data$AYB)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   1754    1917    1931    1937    1949    2018      61 

We find that there are NA's in the AYB column. We replace the NAs with the median of the column

In [ ]:
data$AYB[is.na(housing_price$AYB)] <- median(data$AYB, na.rm = TRUE)

In [6]:
summary(data$YR_RMDL)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
     20    2003    2008    2005    2013    2019   16351 

There are alot of NAs in the YR_RMDL column which indicates that the houses have never been remodelled. For the houses that have never been remodelled, we will replace the NA with the corresponding year in the AYB column.

In [ ]:
data[is.na(data$YR_RMDL),"YR_RMDL"] = data[is.na(data$YR_RMDL),"AYB"]

In [8]:
head(data$SALEDATE,5)

[1] 2010-02-26 00:00:00 2010-11-04 00:00:00 2006-01-09 00:00:00
[4] 2012-08-21 00:00:00 2012-07-25 00:00:00
6181 Levels: 1982-06-11 00:00:00 1984-07-20 00:00:00 ... 2018-07-12 00:00:00

The SALEDATE is made up of the year, month, day and time. We will only make use of the year so we extract only the year and throw everything else out.

In [ ]:
rep = substr(data$SALEDATE,0,4)
data$SALEDATE = factor(rep)
data$SALEDATE = rep
data$SALEDATE = as.numeric(data$SALEDATE)

In [9]:
summary(data$AC)

0     N     Y 
    5  8703 30812

In the AC column, we have 5 values that are "0". It is unclear what these mean but we assume that these are really suppose to be "N". We also order the categories in the AC column with "Y" being the highest category as we expect houses with AC should be more expensive.

In [ ]:
data[data$AC == 0,]$AC = "N"
data$AC <- droplevels(data$AC)
ord = c("N","Y")
data$AC <- ordered(data$AC,levels = ord)

We want to treat the following columns as categorical variables so we convert them to factors

In [ ]:
data$ZIPCODE = factor(data$ZIPCODE)
data$CENSUS_TRACT = factor(data$CENSUS_TRACT)

In [ ]:

names = c("Ward 1","Ward 2","Ward 3","Ward 4","Ward 5","Ward 6","Ward 7","Ward 8")
most_common = c()
for (ward in names){
  d = data.frame(table(housing_price[housing_price$WARD == ward,"QUADRANT"]))
  d$Var1 = levels(droplevels(d$Var1))
  m = max(d$Freq)
  target = subset(d,Freq == m)[1,1]
  most_common = c(most_common,target)
}

quad_df = data.frame(names,most_common)
quad_df = remove.factors(quad_df)

for (i in 1:length(data$QUADRANT)){
  if (is.na(data[i,"QUADRANT"])){
    temp =  subset(quad_df,names == data[i,"WARD"])
    data[i,"QUADRANT"] =temp[1,2]
  }
}

data[data$CNDTN == "Fair","CNDTN"] = "Average"
data$CNDTN <- droplevels(data$CNDTN)
ord = c("Poor","Average","Good","Very Good","Excellent")
data$CNDTN <- ordered(data$CNDTN,levels = ord)


for (i in 1:length(data$GRADE))
{
  if (data[i,"GRADE"] == "Exceptional-B" || data[i,"GRADE"] == "Exceptional-C" || data[i,"GRADE"] == "Exceptional-D"){
    data[i,"GRADE"] = "Exceptional-A"
  }
  else if (data[i,"GRADE"] == "Low Quality" || data[i,"GRADE"] == "Fair Quality"){
    data[i,"GRADE"] = "Average"
  }
  else if (data[i,"GRADE"] == "Good Quality"){
     data[i,"GRADE"] = "Above Average"
  }
}

data$GRADE <- droplevels(data$GRADE)
ord = c("Average","Above Average","Good Quality","Very Good","Excellent","Exceptional-A","Superior")
data$GRADE <- ordered(data$GRADE,levels = ord)

# Replacing the NA in KTICHENS column with the mode
y <- table(data$KITCHENS)
m <- as.numeric(names(y)[which(y==max(y))])
data$KITCHENS[is.na(data$KITCHENS)] = m  


# we fill the NA's in the column with the number from the "Style" column
for (i in 1:length(data$Id)){
  if (is.na(data[i,"STORIES"])){
    data[i,"STORIES"] = as.numeric(word(data[i,"STYLE"],1))
  }
}

data[data$STORIES == 0,]$STORIES = as.numeric(word(data[data$STORIES == 0,]$STYLE,1))

lvl = word(housing_price[!is.na(data$CENSUS_BLOCK),]$CENSUS_BLOCK,2)
lvl = unique(lvl)
levels(data$CENSUS_BLOCK) = c(levels(data$CENSUS_BLOCK),lvl)
data[!is.na(data$CENSUS_BLOCK),]$CENSUS_BLOCK = word(data[!is.na(data$CENSUS_BLOCK),]$CENSUS_BLOCK,2)
data[is.na(data$CENSUS_BLOCK),]$CENSUS_BLOCK = "1001"
data$CENSUS_BLOCK <- droplevels(data$CENSUS_BLOCK)

levels(data$HEAT) = c(levels(data$HEAT),"Other")
for (i in 1:length(data$Id)){
    if (data[i,"HEAT"]!= "Warm Cool" && data[i,"HEAT"]!= "Forced Air" && data[i,"HEAT"] != "Hot Water Rad") {data[i,"HEAT"] = 'Other'}
}
data$HEAT <- droplevels(data$HEAT)

levels(data$STRUCT) = c(levels(data$STRUCT),"Other")
  #combine categories of STRUCT
for (i in 1:length(data$STRUCT))
{
    if (data[i,"STRUCT"]!= "Row Inside" && data[i,"STRUCT"]!= "Row End" && data[i,"STRUCT"]!= "Semi-Detached" && data[i,"STRUCT"]!='Single') {data[i,"STRUCT"] = "Other"}
}
data$STRUCT <- droplevels(data$STRUCT)


  levels(data$ROOF) = c(levels(data$ROOF),"Other")
  #combine categories of ROOF
  for (i in 1:length(data$ROOF))
  {
    if (data[i,"ROOF"]!= "Built Up" && data[i,"ROOF"]!= "Comp Shingle" && data[i,"ROOF"]!= "Metal-Sms" && data[i,"ROOF"]!= "Slate") {data[i,"ROOF"] = 'Other'}
  }

data$ROOF <- droplevels(data$ROOF)

  levels(data$EXTWALL) = c(levels(data$EXTWALL),"Other")
  #combine categories of EXTWALL
  for (i in 1:length(data$EXTWALL))
  {
    if (data[i,"EXTWALL"]!= "Common Brick" && data[i,"EXTWALL"]!= "Brick/Siding" && data[i,"EXTWALL"]!= "Vinyl Siding" && data[i,"EXTWALL"]!= "Wood Siding") {data[i,"EXTWALL"] = 'Other'}
  }

data$EXTWALL <- droplevels(data$EXTWALL)


  levels(data$INTWALL) = c(levels(data$INTWALL),"Other")
  #combine categories of INTWALL
  for (i in 1:length(data$INTWALL))
  {
    if (data[i,"INTWALL"]!= "Hardwood" && data[i,"INTWALL"]!= "Hardwood/Crap" && data[i,"INTWALL"]!= "Wood Floor" && data[i,"INTWALL"]!="Carpet") {data[i,"INTWALL"] = 'Other'}
  }

levels(data$ASSESSMENT_NBHD) = c(levels(data$ASSESSMENT_NBHD),"Other")

#combine categories of INTWALL
  for (i in 1:length(data$ASSESSMENT_NBHD))
  {
    if (data[i,"ASSESSMENT_NBHD"] == "Foggy Bottom" || data[i,"ASSESSMENT_NBHD"] == "Central-tri 1" || data[i,"ASSESSMENT_NBHD"] == "Massachusetts Avenue Heights" ||data[i,"ASSESSMENT_NBHD"] == "Woodley" || data[i,"ASSESSMENT_NBHD"] == "Barry Farms") {data[i,"ASSESSMENT_NBHD"] = 'Other'}
  }

data$ASSESSMENT_NBHD <- droplevels(data$ASSESSMENT_NBHD)

lvl = levels(data$ASSESSMENT_NBHD)
levels(data$ASSESSMENT_SUBNBHD) = c(levels(data$ASSESSMENT_SUBNBHD),lvl)
data[is.na(data$ASSESSMENT_SUBNBHD),"ASSESSMENT_SUBNBHD"] = data[is.na(data$ASSESSMENT_SUBNBHD),"ASSESSMENT_NBHD"] 
data$ASSESSMENT_SUBNBHD <- droplevels(data$ASSESSMENT_SUBNBHD)

data$LANDAREA = (data$LANDAREA - mean(data$LANDAREA))/(sd(data$LANDAREA))
data$GBA =  (data$GBA - mean(data$GBA))/(sd(data$GBA))

data[data$Id == 798,"BATHRM"] <- median(data$BATHRM, na.rm = TRUE)
data[data$Id == 798,"HF_BATHRM"] <- median(data$HF_BATHRM, na.rm = TRUE)
data[data$Id == 798,"BEDRM"] <- median(data$BEDRM, na.rm = TRUE)
data[data$Id == 798,"ROOMS"] <- median(data$ROOMS, na.rm = TRUE)
data[data$Id == 798,"AC"] = "Y"

data$RM = data$BATHRM + data$HF_BATHRM + data$BEDRM + data$ROOMS

data[data$STORIES == 826,]$STORIES = 4
data[data$STORIES == 275,]$STORIES = 3
data[data$STORIES == 250,]$STORIES = 2.5

data = data[data$Id != 12875 & data$Id != 21997,]

testingData = data
drop1 <- c("FULLADDRESS","NATIONALGRID","CENSUS_BLOCK","USECODE")
data = data[ , !(names(data) %in% drop1)]
testingData = testingData[ , !(names(testingData) %in% drop1)]

In [ ]:
df1 = subset(data,data$fold == '1')
df2 = subset(data,data$fold == '2')
df3 = subset(data,data$fold == '3')
df4 = subset(data,data$fold == '4')
df5 = subset(data,data$fold == '5')

drops2 <- c("fold","Id","ROOMS","BEDRM")

train_ex1 = rbind(df2,df3,df4,df5)
train_ex1 = train_ex1[ , !(names(train_ex1) %in% drops2)]
test_1 = subset(testingData,testingData$fold == "1")
test_1 = test_1[ , !(names(test_1) %in% drops2)]
#rf_ex1 = ranger(log(PRICE) ~ . , data = train_ex1,mtry = 13,min.node.size = 1,num.trees =1000)
#pred1 = predict(rf_ex1,data = test_1) # Testing on fold1
out1 = data.frame(testingData[testingData$fold == "1","Id"],pred1$predictions)

train_ex2 = rbind(df1,df3,df4,df5)
train_ex2 = train_ex1[ , !(names(train_ex1) %in% drops2)]
test_2 = subset(testingData,testingData$fold == "2")
test_2 = test_2[ , !(names(test_2) %in% drops2)]
#rf_ex2 = ranger(log(PRICE) ~ . , data = train_ex2,mtry = 13,min.node.size= 1,num.trees = 1000)
#pred2 = predict(rf_ex1,data = test_2) # Testing on fold2
out2 = data.frame(testingData[testingData$fold == "2","Id"],pred2$predictions)

train_ex3 = rbind(df1,df2,df4,df5)
train_ex3 = train_ex3[ , !(names(train_ex3) %in% drops2)]
test_3 = subset(testingData,testingData$fold == "3")
test_3 = test_3[ , !(names(test_3) %in% drops2)]
#rf_ex3 = ranger(log(PRICE) ~ . , data = train_ex3,mtry = 13,min.node.size= 1,num.trees = 1000)
#pred3 = predict(rf_ex3,data = test_3) # Testing on fold3
out3 = data.frame(testingData[testingData$fold == "3","Id"],pred3$predictions)

train_ex4 = rbind(df1,df2,df3,df5)
train_ex4 = train_ex4[ , !(names(train_ex4) %in% drops2)]
test_4 = subset(testingData,testingData$fold == "4")
test_4 = test_4[ , !(names(test_4) %in% drops2)]
#rf_ex4 = ranger(log(PRICE) ~ . , data = train_ex4,mtry = 13,min.node.size= 1,num.trees = 1000)
#pred4 = predict(rf_ex4,data = test_4) # Testing on fold4
out4 = data.frame(testingData[testingData$fold == "4","Id"],pred4$predictions)

train_ex5 = rbind(df1,df2,df3,df4)
train_ex5 = train_ex5[ , !(names(train_ex5) %in% drops2)]
test_5 = subset(testingData,testingData$fold == "5")
test_5 = test_5[ , !(names(test_5) %in% drops2)]
#rf_ex5 = ranger(log(PRICE) ~ . , data = train_ex5,mtry=13,min.node.size= 1,num.trees = 1000)
#pred5 = predict(rf_ex5,data = test_5) # Testing on fold5
out5 = data.frame(testingData[testingData$fold == "5","Id"],pred5$predictions)

names(out1) <- c("Id", "PRICE")
names(out2) <- c("Id", "PRICE")
names(out3) <- c("Id", "PRICE")
names(out4) <- c("Id", "PRICE")
names(out5) <- c("Id", "PRICE")

#kaggle_pred = rbind(out1,out2,out3,out4,out5)
#colnames(kaggle_pred) = c("Id","PRICE")
#kaggle_pred = kaggle_pred[order(kaggle_pred$Id),]
#kaggle_pred$PRICE = exp(kaggle_pred$PRICE)
# Error Calculation
#rmlse(testingData$PRICE,kaggle_pred$PRICE)